In [1]:
#pip install raiwidgets

In [2]:
#pip install --upgrade azureml-core

In [1]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.39.0


In [2]:
from azureml.telemetry import set_diagnostics_collection

set_diagnostics_collection(send_diagnostics=True)

Turning diagnostics collection on. 


In [3]:
import azureml.core
from azureml.core import Workspace
import pandas as pd

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.39.0 to work with mlopsdev


In [4]:
df= pd.read_csv('./Data/titanic.csv')
print(df.shape)
print(df.columns)

(891, 12)
Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


In [5]:
df['Age'] = df.groupby(['Pclass', 'Sex'])['Age'].apply(lambda x: x.fillna(x.median()))
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [6]:
df['Loc']= df['Cabin'].apply(lambda x: x[0] if pd.notnull(x) else 'X')

In [7]:
df.drop(['Cabin', 'Ticket'], axis=1, inplace=True)

In [8]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Loc
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S,X
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S,X
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S,C
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S,X


In [9]:
df.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       2
Loc            0
dtype: int64

In [10]:
df.loc[:,'GroupSize'] = 1 + df['SibSp'] + df['Parch']

In [11]:
df['Embarked'] = df['Embarked'].fillna('S')

In [12]:
LABEL = 'Survived'
columns_to_keep = ['Pclass', 'Sex','Age', 'Fare', 'Embared', 'Deck', 'GroupSize']
columns_to_drop = ['Name','SibSp', 'Parch', 'Survived']
df_train = df
df = df_train.drop(['Name','SibSp', 'Parch', 'PassengerId'], axis=1)

df.head(5)

,Survived,Pclass,Sex,Age,Fare,Embarked,Loc,GroupSize
0,0,3,male,22.0,7.2500,S,X,2
1,1,1,female,38.0,71.2833,C,C,2
2,1,3,female,26.0,7.9250,S,X,1
3,1,1,female,35.0,53.1000,S,C,2
4,0,3,male,35.0,8.0500,S,X,1


In [13]:
import os
script_folder = os.path.join(os.getcwd(), "train_remote")
print(script_folder)
os.makedirs(script_folder, exist_ok=True)

/mnt/batch/tasks/shared/LS_root/mounts/clusters/devbox/code/Users/babal/AMLBook2021/AMLBook2022/Chapter3/train_remote


In [14]:
df.to_csv('./train_remote/titanic.csv')
df.head(2)

,Survived,Pclass,Sex,Age,Fare,Embarked,Loc,GroupSize
0,0,3,male,22.0,7.2500,S,X,2
1,1,1,female,38.0,71.2833,C,C,2


In [15]:
from azureml.core import Dataset

#use default datastore retrieved from the workspace through the AML SDK
default_ds = ws.get_default_datastore()


default_ds.upload_files(files=['./train_remote/titanic.csv'], # Upload the diabetes csv files in /data
                        target_path= 'Titanic-data', # Put it in a folder path in the datastore
                        overwrite=True, # Replace existing files of the same name
                        show_progress=True)
#Create a tabular dataset from the path on the datastore 
dataset = Dataset.Tabular.from_delimited_files(default_ds.path('Titanic-data/titanic.csv'))

# Register the dataset
try:
    tab_data_set = dataset.register(workspace=ws, 
                                name= 'Titanic-tabular-dataset',
                                description='Tintanic data',
                                tags = {'format':'csv'},
                                create_new_version=True)
    print('Dataset registered.')
except Exception as ex:
        print(ex)

"datastore.upload_files" is deprecated after version 1.0.69. Please use "FileDatasetFactory.upload_directory" instead. See Dataset API change notice at https://aka.ms/dataset-deprecation.


Uploading an estimated of 1 files
Uploading ./train_remote/titanic.csv
Uploaded ./train_remote/titanic.csv, 1 files out of an estimated total of 1
Uploaded 1 files
Dataset registered.


In [16]:
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required
from azureml.core import Workspace, Dataset

subscription_id = 'c46a9435-c957-4e6c-a0f4-b9a597984773'
resource_group = 'mlops'
workspace_name = 'mlopsdev'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='Titanic-tabular-dataset')
dataset.to_pandas_dataframe()

,Column1,Survived,Pclass,Sex,Age,Fare,Embarked,Loc,GroupSize
0,0,0,3,male,22.0,7.2500,S,X,2
1,1,1,1,female,38.0,71.2833,C,C,2
2,2,1,3,female,26.0,7.9250,S,X,1
3,3,1,1,female,35.0,53.1000,S,C,2
4,4,0,3,male,35.0,8.0500,S,X,1
...,...,...,...,...,...,...,...,...,...
886,886,0,2,male,27.0,13.0000,S,X,1
887,887,1,1,female,19.0,30.0000,S,B,1
888,888,0,3,female,21.5,23.4500,S,X,4
889,889,1,1,male,26.0,30.0000,C,C,1


In [17]:
from azureml.core.experiment import Experiment
experiment = Experiment(ws, 'titanic_remote_compute')

In [18]:
import os
script_folder = os.path.join(os.getcwd(), "train")
print(script_folder)
os.makedirs(script_folder, exist_ok=True)

/mnt/batch/tasks/shared/LS_root/mounts/clusters/devbox/code/Users/babal/AMLBook2021/AMLBook2022/Chapter3/train


In [19]:
%%writefile $script_folder/training.py

import os
import sys
import argparse
import joblib
import pandas as pd
import numpy as np

from azureml.core import Run, Dataset, Workspace, Experiment

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import roc_auc_score,roc_curve
from interpret.ext.blackbox import TabularExplainer
from azureml.interpret import ExplanationClient

from raiwidgets import ResponsibleAIDashboard
from responsibleai import RAIInsights

# Calculate model performance metrics
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt

from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration

def getRuntimeArgs():
    parser = argparse.ArgumentParser()
    
    parser = argparse.ArgumentParser()
    parser.add_argument("--input-data", type=str)
    args = parser.parse_args()
    return args

def buildpreprocessorpipeline(X_raw):
    categorical_features = X_raw.select_dtypes(include=['object']).columns
    numeric_features = X_raw.select_dtypes(include=['float','int64']).columns

    categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='constant', fill_value="missing")),
                                              ('onehotencoder', OneHotEncoder(categories='auto', sparse=False, handle_unknown='ignore'))])
    numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
    
    preprocessor = ColumnTransformer(
        transformers=[
            ('numeric', numeric_transformer, numeric_features),
            ('categorical', categorical_transformer, categorical_features)
        ], remainder="drop")
    
    return preprocessor

def model_train(LABEL, df, run):  
    y_raw = df[LABEL]
    X_raw = df.drop([LABEL], axis=1)
    
     # Train test split
    X_train, X_test, y_train, y_test = train_test_split(X_raw, y_raw, test_size=0.3, random_state=0)
    
    lg = LogisticRegression(penalty='l2', C=1.0, solver='liblinear')
    preprocessor = buildpreprocessorpipeline(X_train)
    
    #estimator instance
    clf = Pipeline(steps=[('preprocessor', preprocessor),
                               ('regressor', lg)])

    model = clf.fit(X_train, y_train)
    
    
    # calculate AUC
    y_scores = model.predict_proba(X_test)
    auc = roc_auc_score(y_test,y_scores[:,1])
    print('AUC: ' + str(auc))
    run.log('AUC', np.float(auc))

    
    # calculate test accuracy
    y_hat = model.predict(X_test)
    acc = np.average(y_hat == y_test)
    print('Accuracy:', acc)
    run.log('Accuracy', np.float(acc))

    # plot ROC curve
    fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1])
    fig = plt.figure(figsize=(6, 4))
    # Plot the diagonal 50% line
    plt.plot([0, 1], [0, 1], 'k--')
    # Plot the FPR and TPR achieved by our model
    plt.plot(fpr, tpr)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    run.log_image(name = "ROC", plot = fig)
    plt.show()

    # plot confusion matrix
    # Generate confusion matrix
    cmatrix = confusion_matrix(y_test, y_hat)
    cmatrix_json = {
        "schema_type": "confusion_matrix",
           "schema_version": "v1",
           "data": {
               "class_labels": ["0", "1"],
               "matrix": [
                   [int(x) for x in cmatrix[0]],
                   [int(x) for x in cmatrix[1]]
               ]
           }
    }
    
    run.log_confusion_matrix('ConfusionMatrix_Test', cmatrix_json)
    
    os.makedirs('outputs', exist_ok=True)
    
    
    model_file = os.path.join('outputs', 'titanic_model.pkl')
    joblib.dump(value=model, filename=model_file)
    
    run.upload_file(name='titanic_model.pkl', path_or_stream=model_file)
    run.log('accuracy', acc)
    run.set_tags({ 'Accuracy' : np.float(acc)})
    
    # Register the model
    print('Registering model...')
    run.register_model(model_path='titanic_model.pkl', model_name= 'titanic-model',
                   tags={'Model Type':'Logistic Regresssion'},
                   properties={'AUC': np.float(auc), 'Accuracy': np.float(acc)})
    
    #features = "'Column1', 'Pclass', 'Sex', 'Age', 'Fare', 'Embarked', 'Loc', 'GroupSize'"
    #df1 = df.drop('Survived', inplace=true)
    #df1 = df
    columns = ['Survived']
    #df = df.drop('Survived', axis=1, inplace=True)
    df = df.drop(columns, axis=1)
    print(df.columns)
    client = ExplanationClient.from_run(run)
    # Explain predictions on your local machine
    tabular_explainer = TabularExplainer(model, X_train, features=df.columns)

    # Explain overall model predictions (global explanation)
    # Passing in test dataset for evaluation examples - note it must be a representative sample of the original data
    # x_train can be passed as well, but with more examples explanations it will
    # take longer although they may be more accurate
    global_explanation = tabular_explainer.explain_global(X_test)

    # Uploading model explanation data for storage or visualization in webUX
    # The explanation can then be downloaded on any compute
    comment = 'Global explanation on regression model trained on boston dataset'
    client.upload_model_explanation(global_explanation, comment=comment, model_id=model)
    categorical_features = X_raw.select_dtypes(include=['object']).columns
    target_feature = LABEL
    train_data = X_train.copy()
    test_data = X_test.copy()
    train_data[target_feature] = y_train
    test_data[target_feature] = y_test
    #data.feature_names

    rai_insights = RAIInsights(model, train_data, test_data, LABEL, 'classification', 
                               categorical_features=['Sex','Embarked','Loc'])
    # Interpretability
    rai_insights.explainer.add()
    # Error Analysis
    rai_insights.error_analysis.add()
    # Counterfactuals: accepts total number of counterfactuals to generate, the range that their label should fall under, 
    # and a list of strings of categorical feature names
    rai_insights.counterfactual.add(total_CFs=20, desired_class='opposite')
    rai_insights.compute()
    ResponsibleAIDashboard(rai_insights)


    return model, auc, acc
    # Save the trained model
    
    
def main():
    # Create an Azure ML experiment in your workspace
    args = getRuntimeArgs()
    
    run = Run.get_context()
    client = ExplanationClient.from_run(run)

    dataset_dir = './dataset/'
    os.makedirs(dataset_dir, exist_ok=True)
    ws = run.experiment.workspace
    print(ws)
    

    print("Loading Data...")
    dataset = Dataset.get_by_id(ws, id=args.input_data)
    # Load a TabularDataset & save into pandas DataFrame
    df = dataset.to_pandas_dataframe()
    
    print(df.head(5))
 
    model, auc, acc = model_train('Survived', df, run)
    
    #os.makedirs('outputs', exist_ok=True)
    
    
    #model_file = os.path.join('outputs', 'titanic_model.pkl')
    #joblib.dump(value=model, filename=model_file)
    
    #run.upload_file(name='titanic_model.pkl', path_or_stream=model_file)
    #run.log('accuracy', acc)
    #run.set_tags({ 'Accuracy' : np.float(acc)})
    
    # Register the model
    #print('Registering model...')
    #run.register_model(model_path='titanic_model.pkl', model_name= 'titanic-model',
    #               tags={'Model Type':'Logistic Regresssion'},
    #               properties={'AUC': np.float(auc), 'Accuracy': np.float(acc)})
   

    run.complete()

if __name__ == "__main__":
    main()

Overwriting /mnt/batch/tasks/shared/LS_root/mounts/clusters/devbox/code/Users/babal/AMLBook2021/AMLBook2022/Chapter3/train/training.py


In [20]:
%%writefile $script_folder/experiment_env.yml
name: experiment_env
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2
- scikit-learn
- ipykernel
- matplotlib
- pandas
- pip
- pip:
  - azureml-defaults
  - pyarrow
  - interpret
  - azureml-interpret
  - lightgbm
  - raiwidgets

Overwriting /mnt/batch/tasks/shared/LS_root/mounts/clusters/devbox/code/Users/babal/AMLBook2021/AMLBook2022/Chapter3/train/experiment_env.yml


In [21]:
from azureml.core import Environment

# Create a Python environment for the experiment (from a .yml file)
experiment_env = Environment.from_conda_specification('experiment-env', script_folder + "/experiment_env.yml")

# Let Azure ML manage dependencies
experiment_env.python.user_managed_dependencies = False 

# Print the environment details
print(experiment_env.name, 'defined.')
print(experiment_env.python.conda_dependencies.serialize_to_string())

experiment-env defined.
name: experiment_env
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2
- scikit-learn
- ipykernel
- matplotlib
- pandas
- pip
- pip:
  - azureml-defaults
  - pyarrow
  - interpret
  - azureml-interpret
  - lightgbm
  - raiwidgets



In [39]:
import azureml.core.runconfig
from azureml.core import Environment, Experiment
from azureml.core import ScriptRunConfig
from azureml.widgets import RunDetails

# Get the training dataset
titanic_ds = ws.datasets.get('Titanic-tabular-dataset')

# Create a script config
script_config = ScriptRunConfig(source_directory=script_folder,
                                script='training.py',
                                arguments=['--input-data', titanic_ds.as_named_input('titanic')], # Reference to dataset
                                environment=experiment_env) 

# submit the experiment
run = experiment.submit(config=script_config)
RunDetails(run).show()
run.wait_for_completion()

Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': True, 'log_level': 'INFO', 's…

Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "Unable to run conda package manager. AzureML uses conda to provision python\nenvironments from a dependency specification. To manage the python environment\nmanually instead, set userManagedDependencies to True in the python environment\nconfiguration. To use system managed python environments, install conda from:\nhttps://conda.io/miniconda.html",
        "messageParameters": {},
        "details": []
    },
    "time": "2022-04-18T23:23:56.645582Z"
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Activity Failed:\n{\n    \"error\": {\n        \"code\": \"UserError\",\n        \"message\": \"Unable to run conda package manager. AzureML uses conda to provision python\\nenvironments from a dependency specification. To manage the python environment\\nmanually instead, set userManagedDependencies to True in the python env

In [33]:
!python3 --version

Python 3.6.9


Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [22]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "cpu-cluster"

try:
    # Check for existing compute target
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        training_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)

Found existing cluster, use it.


In [23]:
import azureml.core.runconfig
from azureml.core import Environment, Experiment
from azureml.core import ScriptRunConfig
from azureml.widgets import RunDetails

# Get the training dataset
titanic_ds = ws.datasets.get('Titanic-tabular-dataset')

# Create a script config
script_config = ScriptRunConfig(source_directory=script_folder,
                                script='training.py',
                                arguments=['--input-data', titanic_ds.as_named_input('titanic')], # Reference to dataset
                                environment=experiment_env,
                                compute_target=cluster_name)

# submit the experiment
run = experiment.submit(config=script_config)
RunDetails(run).show()

2022/04/19 11:43:20 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2022/04/19 11:43:22 INFO mlflow.tracking.fluent: Autologging successfully enabled for tensorflow.
2022/04/19 11:43:23 INFO mlflow.tracking.fluent: Autologging successfully enabled for lightgbm.


_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': True, 'log_level': 'INFO', 's…

Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [24]:
cluster_state = training_cluster.get_status()
print(cluster_state.allocation_state, cluster_state.current_node_count)

Steady 1


In [25]:
run.wait_for_completion()

Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms

{'runId': 'titanic_remote_compute_1650368584_86e116d0',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-04-19T11:43:20.931495Z',
 'endTimeUtc': '2022-04-19T11:47:22.34185Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'amlctrain',
  'ContentSnapshotId': '9e803317-a7ff-4da6-ae23-75409c7a9617',
  'azureml.git.repository_uri': 'https://github.com/balakreshnan/AMLBook2022.git',
  'mlflow.source.git.repoURL': 'https://github.com/balakreshnan/AMLBook2022.git',
  'azureml.git.branch': 'main',
  'mlflow.source.git.branch': 'main',
  'azureml.git.commit': 'dd916a5609d3c3e8f2e416d70ceb0d3261749477',
  'mlflow.source.git.commit': 'dd916a5609d3c3e8f2e416d70ceb0d3261749477',
  'azureml.git.dirty': 'True',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [{'dataset': {'id': 'cb7282d6-14b7-41f8-b452-4c0343e764c2'}, 'consumptionDetails': {'type': 'RunInput', 'inputName': 'tit

In [26]:
#from azureml.train.hyperdrive import RandomParameterSampling
#from azureml.train.hyperdrive import normal, uniform, choice
#param_sampling = RandomParameterSampling( {
#        "learning_rate": normal(10, 3),
#        "keep_probability": uniform(0.05, 0.1),
#        "batch_size": choice(16, 32, 64, 128)
#    }
#)

In [27]:
#primary_metric_name="accuracy",
#primary_metric_goal=PrimaryMetricGoal.MAXIMIZE

In [28]:
from azureml.core.run import Run
run_logger = Run.get_context()
#run_logger.log("accuracy", float(accuracy))

Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [29]:
#RandomParameterSampling

In [30]:
#from azureml.train.hyperdrive import TruncationSelectionPolicy
#early_termination_policy = TruncationSelectionPolicy(evaluation_interval=1, truncation_percentage=20, delay_evaluation=5)

In [31]:
from azureml.train.hyperdrive import BanditPolicy
early_termination_policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

In [32]:
ds = Dataset.get_by_name(ws, name="Titanic-tabular-dataset")

In [33]:
print(ds.id)

cb7282d6-14b7-41f8-b452-4c0343e764c2


Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [34]:
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.parameter_expressions import choice
    
#arguments=['--input-data', titanic_ds.as_named_input('titanic')], # Reference to dataset

titanic_ds = ws.datasets.get('Titanic-tabular-dataset')

# Create a script config
script_config = ScriptRunConfig(source_directory=script_folder,
                                script='training.py',
                                arguments=['--input-data', titanic_ds.as_named_input('titanic')], # Reference to dataset
                                environment=experiment_env,
                                compute_target=cluster_name) 

#param_sampling = RandomParameterSampling( {
#    "--input-data": ds.id,
#    "batch_size": choice(16, 32, 64, 128),
#    "number_of_hidden_layers": choice(range(1,5))
#    }
#)

param_sampling = RandomParameterSampling( {
    "input-data": ds.id
    }
)

Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [35]:
param_sampling

Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [36]:
early_termination_policy = BanditPolicy(slack_factor=0.15, evaluation_interval=1, delay_evaluation=10)

hd_config = HyperDriveConfig(run_config=script_config,
                             hyperparameter_sampling=param_sampling,
                             policy=early_termination_policy,
                             primary_metric_name="accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=100,
                             max_concurrent_runs=4)

In [37]:
#experiment_name = "titanic_remote_compute"

In [38]:
#from azureml.core.experiment import Experiment
#experiment = Experiment(workspace, experiment_name)
hyperdrive_run = experiment.submit(hd_config)

Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [39]:
from azureml.widgets import RunDetails
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': True, 'log_level': 'INFO',…

Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [40]:
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.parameter_expressions import choice
    
#arguments=['--input-data', titanic_ds.as_named_input('titanic')], # Reference to dataset

titanic_ds = ws.datasets.get('Titanic-tabular-dataset')

# Create a script config
script_config = ScriptRunConfig(source_directory=script_folder,
                                script='training.py',
                                arguments=['--input-data', titanic_ds.as_named_input('titanic')], # Reference to dataset
                                environment=experiment_env,
                                compute_target=cluster_name) 

#param_sampling = RandomParameterSampling( {
#    "--input-data": ds.id,
#    "batch_size": choice(16, 32, 64, 128),
#    "number_of_hidden_layers": choice(range(1,5))
#    }
#)

param_sampling = RandomParameterSampling( {
    "input-data": ds.id
    }
)

early_termination_policy = BanditPolicy(slack_factor=0.15, evaluation_interval=1, delay_evaluation=10)

hd_config = HyperDriveConfig(run_config=script_config,
                             hyperparameter_sampling=param_sampling,
                             policy=early_termination_policy,
                             primary_metric_name="accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=4,
                             max_concurrent_runs=4)
hyperdrive_run = experiment.submit(hd_config)
from azureml.widgets import RunDetails
RunDetails(hyperdrive_run).show()

Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': True, 'log_level': 'INFO',…

Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [44]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['accuracy'])
print('\n learning rate:',parameter_values[3])

#print('\n keep probability:',parameter_values[5])
#print('\n batch size:',parameter_values[7])

Best Run Id:  HD_fdd606ff-965c-4dd9-a5a8-531f0ca95fbe_0

 Accuracy: 0.8097014925373134

 learning rate: cb7282d6-14b7-41f8-b452-4c0343e764c2


In [45]:
parameter_values

['--input-data',
 'DatasetConsumptionConfig:titanic',
 '--input-data',
 'cb7282d6-14b7-41f8-b452-4c0343e764c2']

In [46]:
print(best_run.get_file_names())

['ConfusionMatrix_Test', 'ROC_1650369704.png', 'explanation/302d86d2/eval_data_viz.interpret.json', 'explanation/302d86d2/expected_values.interpret.json', 'explanation/302d86d2/features.interpret.json', 'explanation/302d86d2/global_names/0.interpret.json', 'explanation/302d86d2/global_rank/0.interpret.json', 'explanation/302d86d2/global_values/0.interpret.json', 'explanation/302d86d2/local_importance_values.interpret.json', 'explanation/302d86d2/per_class_names/0.interpret.json', 'explanation/302d86d2/per_class_rank/0.interpret.json', 'explanation/302d86d2/per_class_values/0.interpret.json', 'explanation/302d86d2/rich_metadata.interpret.json', 'explanation/302d86d2/visualization_dict.interpret.json', 'explanation/302d86d2/ys_pred_proba_viz.interpret.json', 'explanation/302d86d2/ys_pred_viz.interpret.json', 'logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/rslex.log', 'logs/azureml/dataprep/rslex.log.2022-04-19

In [47]:
model = best_run.register_model(model_name='titanic-model', model_path='outputs/titanic_model.pkl')

Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [ ]:
script_folder + "/experiment_env.yml"

In [102]:
import os

# Create a folder for the experiment files
deploy_aci_folder = 'deploy-aci'
os.makedirs(deploy_aci_folder, exist_ok=True)
print(deploy_aci_folder, 'folder created')

deploy-aci folder created


In [103]:
model_name = 'titanic-model'
print(model_name)
model = ws.models[model_name]
print(model.name, 'version', model.version)

Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


titanic-model
titanic-model version 69


In [104]:
import joblib
from azureml.core import Dataset
model.download(target_dir='./deploy-aci', exist_ok=True)
dataset = Dataset.get_by_name(ws, name='Titanic-tabular-dataset')
df = dataset.to_pandas_dataframe().head(1)
dftest = df.drop(['Survived'], axis=1)
loaded_model = joblib.load('deploy-aci/titanic_model.pkl')
print('Columns needed for inference:')
print(dftest.columns)
y_hat = loaded_model.predict(dftest)
print('Prediction')
print(y_hat)

Columns needed for inference:
Index(['Column1', 'Pclass', 'Sex', 'Age', 'Fare', 'Embarked', 'Loc',
       'GroupSize'],
      dtype='object')
Prediction
[0]


Trying to unpickle estimator StandardScaler from version 0.24.2 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
Trying to unpickle estimator Pipeline from version 0.24.2 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
Trying to unpickle estimator SimpleImputer from version 0.24.2 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
Trying to unpickle estimator OneHotEncoder from version 0.24.2 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
Trying to unpickle estimator ColumnTransformer from version 0.24.2 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
Trying to unpickle estimator LogisticRegression from version 0.24.2 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
Class

In [105]:
%%writefile $deploy_aci_folder/score.py

import os 
import json
import joblib
from pandas import json_normalize
import pandas as pd

# Called when the service is loaded
def init():
    global model
    # Get the path to the deployed model file and load it
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'titanic_model.pkl')
    model = joblib.load(model_path)

# Called when a request is received
def run(raw_data):
    dict= json.loads(raw_data)
    df = json_normalize(dict['raw_data']) 
    y_pred = model.predict(df)
    print(type(y_pred))
    
    result = {"result": y_pred.tolist()}
    return result

Writing deploy-aci/score.py


In [106]:
%%writefile $deploy_aci_folder/experiment_env.yml
name: experiment_env
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.8
- scikit-learn
- ipykernel
- matplotlib
- pandas
- pip
- pip:
  - azureml-defaults
  - pyarrow

Writing deploy-aci/experiment_env.yml


Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [107]:
from azureml.core import Environment

# Create a Python environment for the experiment (from a .yml file)
experiment_env = Environment.from_conda_specification("experiment-env", deploy_aci_folder + "/experiment_env.yml")

# Let Azure ML manage dependencies
experiment_env.python.user_managed_dependencies = False
# register the environment
experiment_env.register(workspace=ws)
# Print the environment details
print(experiment_env.name, 'defined.')
print(experiment_env.python.conda_dependencies.serialize_to_string())

experiment-env defined.
name: experiment_env
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.8
- scikit-learn
- ipykernel
- matplotlib
- pandas
- pip
- pip:
  - azureml-defaults
  - pyarrow



Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [108]:
from azureml.core.webservice import AciWebservice
aci_config = AciWebservice.deploy_configuration(
            cpu_cores = 1, 
            memory_gb = 2, 
            tags = {'model': 'titanic model'},
            auth_enabled=True,
            enable_app_insights=True,
            collect_model_data=True)

Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [109]:
from azureml.core.model import InferenceConfig
from azureml.core import Model
#get the model object from the workspace
model = Model(ws, model_name)
#get the environment from the workspace
env = Environment.get(ws, 'experiment-env')

inference_config = InferenceConfig(source_directory=deploy_aci_folder,
                                   entry_script='score.py',
                                   environment= env)

# Deploy the model as a service
print('Deploying model...')
service_name = "titanic-service"
service = Model.deploy(ws, service_name, [model], inference_config, aci_config, overwrite=True)

Deploying model...


In [110]:
service.wait_for_deployment(True)
print(service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-04-19 13:38:27+00:00 Creating Container Registry if not exists.
2022-04-19 13:38:27+00:00 Registering the environment.
2022-04-19 13:38:28+00:00 Building image.

Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms

.
2022-04-19 13:44:20+00:00 Generating deployment configuration.
2022-04-19 13:44:21+00:00 Submitting deployment to compute..
2022-04-19 13:44:24+00:00 Checking the status of deployment titanic-service.

Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms

.
2022-04-19 13:46:42+00:00 Checking the status of inference endpoint titanic-service.

Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms


Succeeded


Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


ACI service creation operation finished, operation "Succeeded"
Healthy


In [111]:
endpoint = service.scoring_uri
print(endpoint)
keys = service.get_keys()
selected_key = keys[0]
print(selected_key)

http://6425b711-b5c5-48ab-a5c4-55c5e49e9bce.centralus.azurecontainer.io/score
hReP0Iep5fpIRagd29PgUt7wDyuZrhMP


In [112]:
from azureml.core import Webservice
websrv = Webservice(ws, 'titanic-service')
endpoint = websrv.scoring_uri
print(endpoint)
keys = websrv.get_keys()
selected_key = keys[0]
print(selected_key)

http://6425b711-b5c5-48ab-a5c4-55c5e49e9bce.centralus.azurecontainer.io/score
hReP0Iep5fpIRagd29PgUt7wDyuZrhMP


In [113]:
import json
url = endpoint
api_key = selected_key # Replace this with the API key for the web service
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}
import requests

def MakePrediction(df):
    endpoint_url = url
    body = df.to_json(orient='records') 
    body = '{"raw_data": ' + body + '}'
    print(body)
    r = requests.post(endpoint_url, headers=headers, data=body)
    return (r.json())


dataset = Dataset.get_by_name(ws, name='Titanic-tabular-dataset')
df = dataset.to_pandas_dataframe().head(5)
dftest = df.drop(['Survived'], axis=1)

results = MakePrediction(dftest)

val = results['result']
print('')
print('predictions')
print(val)

{"raw_data": [{"Column1":0,"Pclass":3,"Sex":"male","Age":22.0,"Fare":7.25,"Embarked":"S","Loc":"X","GroupSize":2},{"Column1":1,"Pclass":1,"Sex":"female","Age":38.0,"Fare":71.2833,"Embarked":"C","Loc":"C","GroupSize":2},{"Column1":2,"Pclass":3,"Sex":"female","Age":26.0,"Fare":7.925,"Embarked":"S","Loc":"X","GroupSize":1},{"Column1":3,"Pclass":1,"Sex":"female","Age":35.0,"Fare":53.1,"Embarked":"S","Loc":"C","GroupSize":2},{"Column1":4,"Pclass":3,"Sex":"male","Age":35.0,"Fare":8.05,"Embarked":"S","Loc":"X","GroupSize":1}]}

predictions
[0, 1, 1, 1, 0]


Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [114]:
print(service.state)

Healthy


In [115]:
endpoint = service.scoring_uri
print(endpoint)

http://6425b711-b5c5-48ab-a5c4-55c5e49e9bce.centralus.azurecontainer.io/score


In [116]:
service.delete()

Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [ ]:
#pip install interpret

In [56]:
from interpret import set_visualize_provider
from interpret.provider import InlineProvider
set_visualize_provider(InlineProvider())

Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [57]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

from interpret.glassbox import LogisticRegression
from interpret import show

In [ ]:
seed = 1
X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=seed)

lr = LogisticRegression(random_state=seed)
lr.fit(X_train, y_train)

lr_global = lr.explain_global()
show(lr_global)

lr_local = lr.explain_local(X_test[:5], y_test[:5])
show(lr_local)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
    header=None)
df.columns = [
    "Age", "WorkClass", "fnlwgt", "Education", "EducationNum",
    "MaritalStatus", "Occupation", "Relationship", "Race", "Gender",
    "CapitalGain", "CapitalLoss", "HoursPerWeek", "NativeCountry", "Income"
]
train_cols = df.columns[0:-1]
label = df.columns[-1]
X = df[train_cols]
y = df[label]

seed = 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=seed)

In [ ]:
from interpret.glassbox import ExplainableBoostingClassifier
ebm = ExplainableBoostingClassifier(random_state=seed)
ebm.fit(X_train, y_train)

In [ ]:
from interpret import set_visualize_provider
from interpret.provider import InlineProvider
set_visualize_provider(InlineProvider())

In [ ]:
from interpret import show

ebm_global = ebm.explain_global()
show(ebm_global)

In [1]:
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required
from azureml.core import Workspace, Dataset

subscription_id = 'c46a9435-c957-4e6c-a0f4-b9a597984773'
resource_group = 'mlops'
workspace_name = 'mlopsdev'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='Titanic-tabular-dataset')
dataset.to_pandas_dataframe()

,Column1,Survived,Pclass,Sex,Age,Fare,Embarked,Loc,GroupSize
0,0,0,3,male,22.0,7.2500,S,X,2
1,1,1,1,female,38.0,71.2833,C,C,2
2,2,1,3,female,26.0,7.9250,S,X,1
3,3,1,1,female,35.0,53.1000,S,C,2
4,4,0,3,male,35.0,8.0500,S,X,1
...,...,...,...,...,...,...,...,...,...
886,886,0,2,male,27.0,13.0000,S,X,1
887,887,1,1,female,19.0,30.0000,S,B,1
888,888,0,3,female,21.5,23.4500,S,X,4
889,889,1,1,male,26.0,30.0000,C,C,1
